In [ ]:
# For now, clear outputs (Cell > All Output > Clear) before committing to Git
# There might be a better way

import sqlite3
import pandas as pd
conn = sqlite3.connect('FPA_FOD_20170508.sqlite')

# View all tables in the database
table = pd.read_sql_query("select * from sqlite_master where type = 'table'", conn)
print(table)